In [1]:
pip install -q deepface

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -q image2base64

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -q tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -q confluent-kafka

Note: you may need to restart the kernel to use updated packages.


In [6]:
# imports
from deepface import DeepFace
import os
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt
import uuid
import pandas as pd
from image2base64.converters import base64_to_rgb, rgb2base64
import json
import traceback
import re
import time
import requests
from confluent_kafka import Consumer, Producer
import scipy
import multiprocessing as mp
import dask
from dask import delayed, visualize
from dask.distributed import Client

2023-06-23 13:37:53.255990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# models
path = r"models/EDSR_x2.pb"
sr = cv2.dnn_superres.DnnSuperResImpl_create()
sr.readModel(path)
sr.setModel("edsr", 2)

# directories
db_path = r"stored_faces_db"

# Image resizing options
crop_increase_percent = 25
standard_height = 155
standard_width = 120

# db api
URL_TABLE = "https://gc7da7be5da2e70-g833jueqvvi5nhsa.adb.eu-frankfurt-1.oraclecloudapps.com/ords/admin/persons/"
URL_MODULE = "https://gc7da7be5da2e70-g833jueqvvi5nhsa.adb.eu-frankfurt-1.oraclecloudapps.com/ords/admin/operations/"

# kafka
consumer = Consumer({
    'bootstrap.servers': '192.168.70.40:9092',
    'group.id': 'py',
    'auto.offset.reset': 'earliest'
})
consumer.subscribe(['toAnalyze'])

del_consumer = Consumer({'bootstrap.servers': '192.168.70.40:9092',
    'group.id': 'py',
    'auto.offset.reset': 'earliest'
})
del_consumer.subscribe(['toDelete'])

producer = Producer({'bootstrap.servers': '192.168.70.40:9092'})

# thresholds
face_detection_conf = 0.9
face_recognition_distance = 120

In [8]:
def add_face_to_oracledb(face_dict, cropped_cv2):
    new_id = uuid.uuid4()
    #upscale_start = time.time()
    #upscale_img = upscale(cropped_cv2)
    #print("upscale img takes: " + str(time.time() - upscale_start))
    
    upscale_img = cropped_cv2
    
    data = {
        "in_id": str(new_id),
        "in_age": str(face_dict["age"]),
        "in_gender": face_dict["dominant_gender"],
        "in_emotion": face_dict["dominant_emotion"],
        "in_cropped_img": rgb2base64(upscale_img)
    }

    headers = {'Content-type': 'application/json'}
    response = requests.post(URL_MODULE + "insert_person", json.dumps(data), headers=headers)
    print("Add face to oracle db")

In [9]:
def refresh_face_in_oracledb(face_dict, id, cropped_cv2):
    #upscale_start = time.time()
    #upscale_img = upscale(cropped_cv2)
    #print("upscale img takes: " + str(time.time() - upscale_start))
    
    upscale_img = cropped_cv2
    
    data = {
        "in_id": str(id),
        "in_cropped_img": rgb2base64(upscale_img),
        "in_age": str(face_dict["age"]), 
        "in_gender": face_dict["dominant_gender"], 
        "in_emotion": face_dict["dominant_emotion"]
    }
    
    headers = {'Content-type': 'application/json'}
    response = requests.post(URL_MODULE + "update_person", json.dumps(data), headers=headers)
    print("Refresh face in oracle db")

In [10]:
def handle_db_communication(face_dict, id, cropped_cv2):
    upscale_img = upscale(cropped_cv2)
    
    data = {
        "in_id": str(id),
        "in_cropped_img": rgb2base64(upscale_img),
        "in_age": str(face_dict["age"]), 
        "in_gender": face_dict["dominant_gender"], 
        "in_emotion": face_dict["dominant_emotion"]
    }
    headers = {'Content-type': 'application/json'}
    
    if id is "NEW":
        data["in_id"] = str(uuid.uuid4())
        response = requests.post(URL_MODULE + "insert_person", json.dumps(data), headers=headers)
        print("Add face to oracle db")
    else:
        response = requests.post(URL_MODULE + "update_person", json.dumps(data), headers=headers)
        print("Refresh face in oracle db")

In [11]:
def batch_insert_faces(persons_data):
    headers = {'Content-type': 'application/json'}
    data = json.dumps(persons_data)
    print(data)
    
    try:
        response = requests.post(URL_MODULE + "insert_person", data=data, headers=headers)
        response.raise_for_status()
        print("Insert persons in Oracle DB")
    except requests.exceptions.RequestException as error:
        print("Error:", error)

In [12]:
def reset_is_present():
    response = requests.post(URL_MODULE + "reset_is_present")

In [13]:
def truncate_oracle_db():
    response = requests.delete(URL_MODULE + "truncate_table")
    response.raise_for_status()

In [14]:
def get_standard_size_img(cropped):
    return cv2.cvtColor(cv2.resize(cropped, (standard_width*2, standard_height*2)), cv2.COLOR_RGB2BGR)

In [15]:
def upscale(cropped_cv2):
    sr_upscale_np = sr.upsample(cropped_cv2)
    sr_upscale_np_corrected = cv2.cvtColor(sr_upscale_np, cv2.COLOR_RGB2BGR)

    return Image.fromarray(sr_upscale_np_corrected)

In [16]:
def check_existence(face_list, face_vector):
    if len(face_list) == 0:
        face_list.append({"vector":face_vector, "id":str(uuid.uuid4()), "time":time.time()})
        return "NEW"
    else:
        for index, element in enumerate(face_list):
            dist = scipy.spatial.distance.euclidean(face_vector, element["vector"])
            print("Checking face with list index" + str(index) + ", distance " + str(dist))
            if dist <= face_recognition_distance:
                print("Similiar face found with distance: " + str(dist) + " at index " + str(index))
                element["vector"] = face_vector
                element["time"] = time.time()
                face_list.pop(index)
                face_list.insert(0, element)
                return element["id"]
        print("No face found, new face added to list")
        face_list.insert(0, {"vector":face_vector, "id":str(uuid.uuid4()), "time":time.time()})
        return "NEW"

In [17]:
def create_new_face(analyze_features, cropped_cv2):
    upscale_img = upscale(cropped_cv2)
    return {
        "id": str(uuid.uuid4),
        "age": analyze_features["age"],
        "gender": analyze_features["dominant_gender"],
        "emotion": analyze_features["dominant_emotion"],
        "cropped_img": rgb2base64(upscale_img)
    }

In [18]:
def process_face(face):
    
    cropped = face["cropped"]
    
    analyze_start = time.time()
    analyze = DeepFace.analyze(img_path=cropped, actions=['age', 'gender','emotion'], 
                           detector_backend='skip', enforce_detection=False, silent=True)
    print("Analyzing 1 person: " + str(time.time()-analyze_start) + " secs")
    recog_start = time.time()
    emb = DeepFace.represent(img_path=cropped, detector_backend= 'skip', model_name="VGG-Face", 
                             enforce_detection=False)[0]["embedding"]
    result = check_existence(face_list, emb) 
    print("Face recognition 1 person: " + str(time.time()-recog_start) + " secs")

    oracle_start = time.time()
    w_increase = int(face['facial_area'].get('w') * crop_increase_percent / 100)
    h_increase = int(face['facial_area'].get('h') * crop_increase_percent / 100)
    x_new = x - w_increase // 2
    y_new = y - h_increase // 2
    w_new = w + w_increase
    h_new = h + h_increase
    
    cropped = get_standard_size_img(img[y_new:y_new + h_new, x_new:x_new + w_new, :])
    
    if result == "NEW":
        add_face_to_oracledb(face_dict=analyze[0], cropped_cv2=cropped)
    else:
        refresh_face_in_oracledb(face_dict=analyze[0], id=result, cropped_cv2=cropped)
    print("Oracle db actions: " + str(time.time()-oracle_start) + " secs")

In [27]:
face_list = []
num_workers = 10  # Specify the desired number of workers
client = Client(n_workers=num_workers)
truncate_oracle_db()

while True:
    try:
        del_msg = del_consumer.poll(1.0)
        
        if del_msg is not None:
            id = del_msg.value().decode('utf-8')
            for face in face_list:
                if face["id"] == id:
                    face_list.remove(face)
                    print("Face deleted")
                    break
      
        msg = consumer.poll(1.0)

        if msg is None:
            print("Waiting")
            continue
        if msg.error():
            print("Consumer error: {}".format(msg.error()))
            continue
        else:
            start = time.time()
            reset_is_present()
            #dict = json.loads(msg.value())
            #for base64 in dict["screens"]:
            
            base64 = msg.value()
            img = base64_to_rgb(base64, 'cv2')
            print("IMG SIZE: " + str(img.shape))
            start = time.time()
            extract_faces_start = time.time()
            faces = DeepFace.extract_faces(img_path=img, detector_backend="mtcnn", 
                                         enforce_detection=False)
            for face in faces:
                #if face["confidence"] < face_detection_conf:
                #    faces.remove(face)

                x = face['facial_area'].get('x')
                y = face['facial_area'].get('y')
                w = face['facial_area'].get('w')
                h = face['facial_area'].get('h')

                #w_increase = int(w * crop_increase_percent / 100)
                #h_increase = int(h * crop_increase_percent / 100)

                #x_new = x - w_increase // 2
                #y_new = y - h_increase // 2
                #w_new = w + w_increase
                #h_new = h + h_increase

                cropped = img[y:y+h, x:x+w, :]
                #cropped = get_standard_size_img(
                #    img[y_new:y_new + h_new, x_new:x_new + w_new, :])

                face["cropped"] = cropped 
                print(face["cropped"].shape)

            computations = [process_face(face) for face in faces]
            dask.compute(*computations)
                
            print("Time elapsed: " + str(time.time()-start))

            #producer.produce(topic="update_wall", value="DB refreshed")
            #producer.flush()
            #producer.produce(topic="update_feeds", value="DB refreshed")
            #producer.flush()
    except Exception as e:
        traceback.print_exc()
        continue
client.close()

Process Dask Worker process (from Nanny):
2023-06-23 13:52:35,085 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/distributed/process.py", line 202, in _run
    target(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/distributed/nanny.py", line 999, in _run
    asyncio.run(run())
  File "/opt/conda/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 636, in run_until_complete
    self.run_forever()
  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  Fi

IMG SIZE: (720, 1280, 3)
1/1 [==============================] - 0s 24ms/step
(88, 65, 3)
(156, 108, 3)
Analyzing 1 person: 0.3116486072540283 secs
Face recognition 1 person: 0.13958096504211426 secs
Add face to oracle db
Oracle db actions: 0.2775123119354248 secs
Analyzing 1 person: 0.2780907154083252 secs
Checking face with list index0, distance 28.50797100223663
Similiar face found with distance: 28.50797100223663 at index 0
Face recognition 1 person: 0.1295170783996582 secs
Refresh face in oracle db
Oracle db actions: 0.44907069206237793 secs
Time elapsed: 2.370495557785034
IMG SIZE: (720, 1280, 3)
2/2 [==============================] - 0s 6ms/step
(720, 1280, 3)
Analyzing 1 person: 0.27812838554382324 secs
Checking face with list index0, distance 89.01730674218747
Similiar face found with distance: 89.01730674218747 at index 0
Face recognition 1 person: 0.12035059928894043 secs
Refresh face in oracle db
Oracle db actions: 0.17032647132873535 secs
Time elapsed: 1.2658350467681885
IM

Traceback (most recent call last):
  File "/tmp/ipykernel_102/2217110441.py", line 63, in <module>
    computations = [process_face(face) for face in faces]
  File "/tmp/ipykernel_102/2217110441.py", line 63, in <listcomp>
    computations = [process_face(face) for face in faces]
  File "/tmp/ipykernel_102/266795043.py", line 23, in process_face
    cropped = get_standard_size_img(img[y_new:y_new + h_new, x_new:x_new + w_new, :])
  File "/tmp/ipykernel_102/2936904325.py", line 2, in get_standard_size_img
    return cv2.cvtColor(cv2.resize(cropped, (standard_width*2, standard_height*2)), cv2.COLOR_RGB2BGR)
cv2.error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



IMG SIZE: (720, 1280, 3)
1/1 [==============================] - 0s 15ms/step
(720, 1280, 3)
Analyzing 1 person: 0.29279112815856934 secs
Checking face with list index0, distance 104.30944079923053
Similiar face found with distance: 104.30944079923053 at index 0
Face recognition 1 person: 0.127211332321167 secs
Refresh face in oracle db
Oracle db actions: 0.16815710067749023 secs
Time elapsed: 1.3576099872589111
IMG SIZE: (720, 1280, 3)
1/1 [==============================] - 0s 19ms/step
(185, 141, 3)
Analyzing 1 person: 0.3279731273651123 secs
Checking face with list index0, distance 109.37204231767423
Similiar face found with distance: 109.37204231767423 at index 0
Face recognition 1 person: 0.11488795280456543 secs
Refresh face in oracle db
Oracle db actions: 0.17002058029174805 secs
Time elapsed: 1.3183379173278809
IMG SIZE: (720, 1280, 3)
1/1 [==============================] - 0s 15ms/step
(720, 1280, 3)
Analyzing 1 person: 0.2844967842102051 secs
Checking face with list index0, di

Process Dask Worker process (from Nanny):
2023-06-23 14:04:56,845 - distributed.nanny - ERROR - Worker process died unexpectedly
2023-06-23 14:04:56,846 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2023-06-23 14:04:56,846 - distributed.nanny - ERROR - Worker process died unexpectedly
2023-06-23 14:04:56,846 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2023-06-23 14:04:56,846 - distributed.nanny - ERROR - Worker process died unexpectedly
2023-06-23 14:04:56,846 - distributed.nanny - ERROR - Worker process died unexpectedly
2023-06-23 14:04:56,846 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2023-06-23 14:04:56,846 - distributed.nanny - ERROR - W

KeyboardInterrupt: 